In [1]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient() # Creates a client that uses the default port on localhost.
database = client.medical_notes_kaggle # Connect to medical_notes_kaggle database

In [2]:
# import data
mongodb_query = database.text.find({})
text_tuple = list(mongodb_query)

In [3]:
text_df = pd.DataFrame.from_dict(text_tuple) # convert to dataframe

In [9]:
text_df.sort_values('index_').head() # ready for modeling

,1. RIGHT BICEPS,10. ANTERIOR TIBIALIS,11. RIGHT GASTROCNEMIUS,12. LEFT GASTROCNEMIUS,13. LEFT VASTUS MEDIALIS,2-D ECHOCARDIOGRAM,2-D M-MODE,2-D STUDY,2. RIGHT TRICEPS,3-DIMENSIONAL SIMULATION,...,WORKUP,WOUND CLOSURE,X-RAY,X-RAY AT NEXT VISIT,X-RAY INTERPRETATION,X-RAYS,_id,clinical_domain,index_,text_0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5d684aa64b8c8ac8cb94f697,Neurology,1001,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5d684aa64b8c8ac8cb94f691,Orthopedic,1002,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5d684aa64b8c8ac8cb94f6a4,Orthopedic,1003,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5d684aa64b8c8ac8cb94f690,Radiology,1004,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,X-rays taken today; three views to the right f...,NaN,5d684aa64b8c8ac8cb94f692,Orthopedic,1005,NaN
